# 2.4.3 Compare Group individual models Version 2 with format updated

In [1]:
import numpy as np
import pandas as pd
import torch
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
import sys
import numpy as np
from numpy.random import seed
### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow as tf


import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Input, add
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import load_model, save_model, Model


from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *


tf.__version__

'2.3.0'

# General Settings

In [3]:

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
seed(random_seed)
tf.random.set_seed(random_seed)
np.random.seed(random_seed)



In [4]:
def seed_everything(seed= 1000):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(seed= random_seed)  

In [5]:
def acti_model():
        model = Sequential()
        model.add(Conv1D(10, 44, strides=2,activation='relu', input_shape=(winlength, 6)))
        model.add(Conv1D(10, 20, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(Conv1D(10, 4, strides=2, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01)))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])


def residual_module(layer_in, n_filters):
            merge_input = layer_in
            # check if the number of filters needs to be increase, assumes channels last format
            if layer_in.shape[-1] != n_filters:
                merge_input = Conv1D(n_filters, (1,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l1(0.01))(layer_in)
            # conv1
            # L2 for avoiding overfitting
            conv1 = Conv1D(n_filters, (5,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01))(layer_in)
            # Batch normal for re-scaling data and better fit data
            conv1 = tf.keras.layers.BatchNormalization(1)(conv1)
            conv1 = tf.keras.layers.ReLU()(conv1)
            # conv2
            conv2 = Conv1D(n_filters, (5,), padding='same', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01))(conv1)
            conv2 = tf.keras.layers.BatchNormalization(1)(conv2)
            
            # add filters, assumes filters/channels last
            #out = conv2
            out = add([conv2, merge_input])
            # activation function
            layer_out = Activation('relu')(out)
            return layer_out
        

def res_model(input_shape):
            in_layer = Input(shape = input_shape)
            out = Conv1D(10, 40, strides=2)(in_layer)
            out = residual_module(layer_in= out,  n_filters=10)
            out = residual_module(layer_in= out,  n_filters=10)
            out = GlobalAveragePooling1D()(out)
            # L1 for feature selection
            out = Dense(200, activation='relu',kernel_regularizer=tf.keras.regularizers.l1(0.01))(out)
            out = Dense(1, activation='sigmoid')(out)
            model = Model(inputs=in_layer, outputs=out)
            return model
        
# newmodel = my_model(input_shape =(900,6) )

# newmodel.compile(loss='binary_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])


In [6]:

from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *
def train_models(model, win_ls = [],EPOCHS = 10,stridesec = 1,name = "wenkanw",model_name="v2" ,random_seed= 1000, split_day=False):
    from numpy.random import seed
    seed(random_seed)
    random.seed(random_seed)
#     tf.set_random_seed(random_seed)
    from datetime  import datetime
    batch_size = 128
    outfile = sys.stdout
    perf = {"model":[],"win(sec)":[], "acc":[],"recall":[], "auc":[]}
    model_ls = []
    hist_ls = []
    for winsize in win_ls:
        tf.random.set_seed(random_seed)
        seed(random_seed)
        
        winmin = winsize
        winlength = int(winmin * 60 * 15)
        step = int(stridesec * 15)
        start_time = datetime.now()
        arr = ["echo -n 'PBS: node is '; cat $PBS_NODEFILE",\
              "echo PBS: job identifier is $PBS_JOBID",\
              "echo PBS: job name is $PBS_JOBNAME"]
        [os.system(cmd) for cmd in arr]
        print("*****************************************************************\n", file=outfile, flush=True)
        print("Execution Started at " + start_time.strftime("%m/%d/%Y, %H:%M:%S"), file=outfile, flush=True)
        print("WindowLength: {:.2f} min ({:d} datum)\tSlide: {:d} ({:d} datum)\tEpochs:{:d}\n".format(winmin, winlength, stridesec, step, EPOCHS), file=outfile, flush=True)


        pathtemp = "../models/" + name+"_models" +"/"+model_name+"_M_F_"
        #pathtemp = "../models/" + name +"/"+model_name+"_M_F_"
        modelpath = pathtemp + "{:f}Min.h5".format(winmin)
        jsonpath = pathtemp + "{:f}Min.json".format(winmin)
        print("Model to Save: ",modelpath)
        print()
        # Load the dataset
        
        person = name
        if split_day:
            create_train_test_file_list(file_name= "all_files_list.txt",person_name =name,
                         out_path = "../data-file-indices/",root_path= "../",
                         test_ratio = 0.2, print_flag = True, shuffle=True, random_state=random_seed)

            meal_data_train = Person_MealsDataset(person_name= person, file_name = "train_files", winmin = winmin,stridesec = stridesec)
            meal_data_test = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = winmin,stridesec = stridesec)

            train_indices, valid_indices = split_train_test_indices(X= [i for i in range(len(meal_data_train.labels))],
                                                                    y = meal_data_train.labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            #balanced train set
            trainset_labels = meal_data_train.labels[train_indices]
            train_indices = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)

            # balance test set
            testset_labels = meal_data_test.labels
            test_indices = balance_data_indices(testset_labels,data_indices=[i for i in range(len(meal_data_test))] ,mode="under", shuffle=True,random_state = random_seed,replace= False)
            # get numpy dataset
            balancedData, balancedLabels = meal_data_train.get_subset(train_indices)
            valid_balancedData, valid_balancedLabels = meal_data_train.get_subset(valid_indices)
            test_Data, test_Labels = meal_data_test.get_subset(test_indices)

        else:
        
            meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
            samples,labels =  meal_data.data_indices, meal_data.labels
            # split train set and test set
            train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                                    y = labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            # balance train set
            trainset_labels = labels[train_indices]
            train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)
            #balance test set
            testset_labels = labels[test_indices]
            test_indices = balance_data_indices(testset_labels,data_indices= test_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)

            train_set_balanced = torch.utils.data.Subset(meal_data, train_indices_balanced)
            test_set = torch.utils.data.Subset(meal_data, test_indices)

            train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
            test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

            print("Data Loader Created")            
            
            # split validation set
            balanced_trainset_labels = labels[train_indices_balanced]
            train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                                    y = balanced_trainset_labels, test_size = 0.2,
                                                                   random_seed = random_seed)
            valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
            valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

            balancedData, balancedLabels = meal_data.get_subset(train_indices)
            valid_balancedData, valid_balancedLabels = meal_data.get_subset(valid_indices)
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        

        #training settings
        mcp_save = tf.keras.callbacks.ModelCheckpoint(modelpath, save_best_only=True, monitor='accuracy')
        

        scheduler = tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.1, patience=3, verbose=0,
                                             mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.)
        #train model
        H = model.fit(x=balancedData, y = balancedLabels,
                       validation_data=(valid_balancedData, valid_balancedLabels),
                    epochs = EPOCHS, batch_size=batch_size, verbose=1,
                    callbacks=[mcp_save,scheduler]) # removed addons.LossHistory(jsonpath) for compatibility with TensorFlow 2.2.0, needs to be re-added at some point

        print("Max value: ", max(H.history['accuracy']), " at epoch", np.argmax(H.history['accuracy']) + 1)

        from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
        predictions = model.predict(x=test_Data)
        threshold = 0.5
        acc =  accuracy_score(predictions>=threshold,test_Labels)
        recall = recall_score(predictions>=threshold,test_Labels)
        auc = roc_auc_score(predictions>=threshold,test_Labels)
        print("Test Accuracy:", acc)
        print("Recall Accuracy:", recall)
        print("AUC Score:", auc)

        perf["model"].append("ActiModel")
        perf["win(sec)"].append(winmin*60)
        perf["acc"].append(acc)
        perf["recall"].append(recall)
        perf["auc"].append(auc)
        model_ls.append(model)
        hist_ls.append(H)
    perf_df = pd.DataFrame(perf)
    print(perf_df)
    return perf_df, model_ls, hist_ls
    

In [7]:
# winmin= 1
# win_size = 15*winmin*60
# model_1 = res_model(input_shape =(900,6) )
# model_1.compile(loss='binary_crossentropy',
#                 optimizer='adam', metrics=['accuracy'])
# perf_df_1, model_ls_1, hist_ls_1 = train_models(model_1,win_ls = [1],EPOCHS = 30,stridesec = 5,model_name="resnet",name = "wenkanw")

In [8]:
from collections import defaultdict

def test_models(winmin=1, stridesec = 5,names= ["wenkanw"],random_seed=1000, split_day=False):
    perf = defaultdict(list)
    meal_info = defaultdict(list)
    for name in names:
        person = name
        if split_day:
            meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = winmin,stridesec = stridesec)

            # balance test set
            testset_labels = meal_data.labels
            test_indices = balance_data_indices(testset_labels,data_indices=[i for i in range(len(meal_data))] ,mode="under", shuffle=True,random_state = random_seed,replace= False)
            # get numpy dataset
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        else:            
            meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
            samples,labels =  meal_data.data_indices, meal_data.labels
            # split train set and test set
            train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                                            y = labels, test_size = 0.2,
                                                                           random_seed = random_seed)
            testset_labels = labels[test_indices]
            test_indices = balance_data_indices(testset_labels,data_indices= test_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)
            test_Data, test_Labels = meal_data.get_subset(test_indices)
        
        meal_counts, min_counts,hour_counts, day_counts = meal_data.get_mealdataset_info(person_name=name)
        meal_info["dataset"].append(name)
        meal_info["Days"].append(day_counts)
        meal_info["Meal_Hours"].append(round(hour_counts,1)) 
        meal_info["Meal_Minutes"].append(round(min_counts,1))
        meal_info["Meal_Counts"].append(meal_counts) 
        perf["dataset"].append(name)
        perf["win(sec)"].append(winmin*60)
        
        
        from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
        group_model_W  = tf.keras.models.load_model('../models/CAD_models/resnet_M_F_1.000000Min.h5')
        group_model_S  = tf.keras.models.load_model('../models/ActiModels/CAD_M_F_1.000000Min.h5')
        individual_model = tf.keras.models.load_model('../models/'+ name+ '_models/resnet_M_F_1.000000Min.h5')
        models = {"suffix":['Individual-Model','W-GroupModel',"S-GroupModel"],  "model":[individual_model,group_model_W,group_model_S]}
        
        for i in range(len(models["suffix"])):
            suffix = models["suffix"][i]
            model = models["model"][i]
            # if the dataset is CAD group dataset and model is individual model
            # we don't need to make prediction on that data
            if name == "CAD" and suffix =='Individual-Model':
                acc = None
                auc = None
                recall = None
            else:

                predictions = model.predict(x=test_Data)
                threshold = 0.5
                acc =  accuracy_score(predictions>=threshold,test_Labels)
                recall = recall_score(predictions>=threshold,test_Labels)
                auc = roc_auc_score(predictions>=threshold,test_Labels)

                acc = round(acc,5)
                recall = round(recall, 5)
                auc = round(auc,5)
            
            print("Test Accuracy:", acc)
            print("Recall Accuracy:", recall)
            print("AUC Score:", auc)
            perf["Acc: "+suffix].append(acc)
            perf["Recall: "+suffix].append(recall)
            perf["Auc: "+suffix].append(auc)

    meal_info = pd.DataFrame(meal_info)
    perf_df = pd.DataFrame(perf)
    return meal_info, perf_df

In [23]:
meal_info, performance = test_models(winmin=1, stridesec = 5,names= ["wenkanw","adam","lawler","shaurya","CAD"],random_seed=1000, split_day=False)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-12-20/10-12-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-27-20/9-27-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-16-20/11-16-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-10-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-3-20/10-3-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Lunch/Lunch.shm
Loading File:  ../data/In

In [24]:
meal_info

,dataset,Days,Meal_Hours,Meal_Minutes,Meal_Counts
0,wenkanw,17,7.7,461.5,32
1,adam,20,4.6,278.3,36
2,lawler,23,25.9,1554.0,61
3,shaurya,6,3.9,236.3,17
4,CAD,354,255.9,15352.6,1063


In [25]:
performance

,dataset,win(sec),Acc: Individual-Model,Recall: Individual-Model,Auc: Individual-Model,Acc: W-GroupModel,Recall: W-GroupModel,Auc: W-GroupModel,Acc: S-GroupModel,Recall: S-GroupModel,Auc: S-GroupModel
0,wenkanw,60,0.91516,0.92910,0.91560,0.67329,0.85165,0.73331,0.73105,0.85854,0.76449
1,adam,60,0.95958,0.94109,0.96039,0.83084,0.92500,0.84792,0.80015,0.88932,0.81677
2,lawler,60,0.83532,0.88058,0.84013,0.66331,0.71956,0.67478,0.66514,0.69556,0.66924
3,shaurya,60,0.83834,0.91361,0.84993,0.65283,0.89140,0.74318,0.67933,0.83498,0.72871
4,CAD,60,NaN,NaN,NaN,0.77677,0.85018,0.78949,0.75852,0.79395,0.76233


In [17]:
# from collections import defaultdict
# dic = defaultdict(list)
# dic['a'].append(None)
# dic['a'].append(1)
# dic['a'].append(2)
# dic_pd = pd.DataFrame(dic)
# dic_pd['a']

In [9]:
# ! cat ../data-file-indices/CAD/all_files_list.txt

In [42]:
# import logging
# import tensorflow as tf
# from tensorflow.compat.v1 import graph_util
# from tensorflow.python.keras import backend as K
# from tensorflow import keras

# # necessary !!!
# tf.compat.v1.disable_eager_execution()
# # K.set_session()
# # K.get_session().close()
# K.set_session(tf.compat.v1.Session())
# # K.get_session().run(tf.global_variables_initializer())
# # model_path = '../models/ActiModels/CAD_M_F_1.000000Min.h5'
# model_path = '../models/wenkanw_models/resnet_M_F_1.000000Min.h5'
# model = keras.models.load_model(model_path)
# model.summary()
# # save pb
# with K.get_session() as sess:
#     output_names = [out.op.name for out in model.outputs]
#     input_graph_def = sess.graph.as_graph_def()
#     for node in input_graph_def.node:
#         node.device = ""
#     graph = graph_util.remove_training_nodes(input_graph_def)
#     graph_frozen = graph_util.convert_variables_to_constants(sess, graph, output_names)
#     tf.io.write_graph(graph_frozen, './log/','./model.pb', as_text=False)
# logging.info("save pb successfully！")

In [9]:
model_path = '../models/wenkanw_models/resnet_M_F_1.000000Min.h5'
model = keras.models.load_model(model_path)
model.save("./log/wenkanw_model2.pb")

INFO:tensorflow:Assets written to: ./log/wenkanw_model2.pb/assets


In [9]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
# time_series = np.random.random(10000)

# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5)
meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
                 normalize_flag = 0)
time_series = meal_data.data[0].T

# Remove outliers
# for i in range(len(time_series)):
#     med = np.quantile(time_series[i],[0.5])
#     boundary = np.quantile(time_series[i],[0.01,0.99])
#     time_series[i][time_series[i]>boundary[1]] = med
#     time_series[i][time_series[i]<boundary[0]] = med
    
def f(offset,winsize,moment):
    if moment!= None:
        offset = int(moment)
    if offset < time_series.shape[1]-winsize:
        t = np.arange(start = offset, stop= offset+winsize, step=1)
        fig, ax = plt.subplots(3,1,figsize= (20,10))
        df1 = time_series[3, offset: offset+winsize]
        df2 = time_series[4, offset: offset+winsize]
        df3 = time_series[5, offset: offset+winsize]
        # set y-axis center at 0
        ax[0].set_ylim(-max(abs(df1)),max(abs(df1)))
        ax[1].set_ylim(-max(abs(df2)),max(abs(df2)))
        ax[2].set_ylim(-max(abs(df3)),max(abs(df3)))
        
        _ = sns.lineplot(t,df1 , ax =ax[0])
        _ = sns.lineplot(t, df2, ax =ax[1])
        _ = sns.lineplot(t, df3, ax =ax[2])
        
    
    return 
interact(f, offset=(0, time_series.shape[1]), winsize= [15*30], moment = '0');

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


interactive(children=(IntSlider(value=258711, description='offset', max=517423), Dropdown(description='winsize…

In [80]:
meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
                 normalize_flag = 0)


Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


In [97]:
df = meal_data.data[0].T
num = 2
np.max(df[num]), np.min(df[num]),len(df[df>30]),len(df[df<-20])

(2.3961902, -2.0423539, 85189, 117751)

In [102]:
num = 3
np.quantile(df[num],[0,0.02,0.25,0.5,0.75,0.98,1])

array([-4.11059570e+02, -4.56250153e+01, -2.53175354e+00,  1.21917725e-02,
        2.71726990e+00,  5.33414972e+01,  6.80175781e+02])

In [20]:
meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 1,
                 normalize_flag = 1)

Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/9-17-20/9-17-20.shm


In [27]:
day  = 0
day1_data = meal_data.data_indices[meal_data.data_indices[:,0]==day]
day1_label = meal_data.labels[meal_data.data_indices[:,0]==day]
day1_data, day1_label

(array([[     0,      0,    900,  42966],
        [     0,     75,    975,  42966],
        [     0,    150,   1050,  42966],
        ...,
        [     0, 516300, 517200,  42966],
        [     0, 516375, 517275,  42966],
        [     0, 516450, 517350,  42966]]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [28]:
len(meal_data.data_indices),len( meal_data.labels),set(day1_label)

(20266, 20266, {0, 1})

In [31]:
subdataset = meal_data.get_dataset(0, len(day1_data))

100%|██████████| 6887/6887 [00:16<00:00, 408.38it/s]


In [37]:
subdataset['data'].iloc[0]

tensor([[-0.8596,  1.9263,  1.1254, -0.0842,  0.1248, -3.1173],
        [-1.0928,  2.8200, -1.6128, -3.7340, -0.8169, -2.9935],
        [-0.4306,  4.1507, -1.5634,  0.4042,  0.2284, -1.2990],
        ...,
        [ 0.1852,  1.2797,  1.0220, -1.7421,  0.5912, -0.3239],
        [ 0.1933,  1.5122,  1.3155, -1.6786,  0.4341, -1.1580],
        [ 0.2658,  1.8572,  1.5431, -1.4450,  0.1537, -1.1859]])

In [35]:
sample, label = meal_data[0][0].numpy(),meal_data[0][1]

model = tf.keras.models.load_model('../models/'+ 'wenkanw'+ '_models/resnet_M_F_1.000000Min.h5')
s2 = np.expand_dims(sample,axis=0)
sample.shape, s2.shape, s2[0].all()==sample.all()
model(s2).numpy()[0][0],label

(4.8908533e-10, 0)

In [60]:
# for i in set(meal_data.data_indices[:,0]):
#     print(i)
# day=1
# np.where(meal_data.data_indices[:,0]==day)

(array([ 6887,  6888,  6889, ..., 13176, 13177, 13178]),)

In [105]:

def hystersis_threshold(model, data,start_threshold=0.8, end_threshold=0.3, winmin = 1, stepsec=5, episode_min = 1.):
    """
    model: tensorflow model
    data:  This dataset must be the self-defined class of Person_MealsDataset  datasetset in my dataset.py/pytorch dataset without using shuffle. 
    Keep the order of dataset after extracting window samples!  You can also define your own dataset using class object to create the interface
    
    start_threshold: the high threshold of the beginning of segmentation
    
    end_threshold: the end threshold of the end of segmentation
    
    winmin: size of a window sample in unit of  minute
    
    stepsec: stride to move the window in unit of second / the number of second between two adjacent window samples
    
    episode_min: the minimum length of eating episode in unit of minute. If end of segmentation -start of segmentation < episode_min,
        then the episode will not be counted
    
    """
    result_ls = []
    t_pause = 100
    
    days = set(data.data_indices[:,0])
    for day in days:
        # Select and Extract the data and labels of the corresponding day from the whole dataset
        sample_indices= np.where(data.data_indices[:,0]==day)[0]
        result = {'day':day, 'segment_start':[], 'segment_end':[],'proba':[],'predictions':np.zeros([len(sample_indices)]),'labels':[],"segment_count":0}
        
        # get the numpy array of samples and labels
        samples, labels = data.get_subset(sample_indices)
        probas = model(samples)
        state = 0
        start = 0
        end = 0 
        pause_counter = 0
        # one day data
        print("Day: ",day)
        for i in range(len(sample_indices)):
            #print("i:",i)
            #sample, label = data[i][0].numpy(),data[i][1]
            #sample = np.expand_dims(sample,axis=0)
            #proba = model(sample).numpy()[0][0]
            sample = samples[i]
            label = labels[i]
            proba = probas[i].numpy()[0]
            
            result['proba'].append(proba)
            result['labels'].append(label)
            
            if state ==0 and proba > start_threshold:
                state = 1
                start = i
            elif state == 1 and proba <end_threshold:
                state = 2
                end = i+1
                pause_counter = 0
            elif state ==2:
                if proba > start_threshold:
                    state = 1
                else:
                    pause_counter += stepsec
                    if pause_counter >= t_pause:
                        # convert time to second and check threshold
                        if (end-start)*stepsec >= episode_min*60:
                            # save data
                            result['segment_start'].append(start)
                            result['segment_end'].append(end)
                            result['segment_count'] += 1
                            result['predictions'][start:end] = 1
                            pass
                        end = 0
                        state = 0
        if state != 0:
            # if segment ended at the end of data
            if end != 0:
                result['segment_start'].append(start)
                result['segment_end'].append(end)
                result['predictions'][start:end] = 1
            else:
                result['segment_count'] -= 1  
            result['segment_count'] += 1
            
        result_ls.append(result)
        print("Segmentation Completed. ")
                            
    return pd.DataFrame(result_ls)

In [86]:
person = 'lawler'
meal_data = Person_MealsDataset(person_name= person, file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 1,
                 normalize_flag = 1)
model = tf.keras.models.load_model('../models/'+ person+ '_models/resnet_M_F_1.000000Min.h5')


Loading Dataset ...
Loading File:  ../data/IndividualData/lawler-data/10.19/10.19.shm
Loading File:  ../data/IndividualData/lawler-data/10.16/10.16.shm
Loading File:  ../data/IndividualData/lawler-data/9.23/9.23_13hr.shm
Loading File:  ../data/IndividualData/lawler-data/10.20/afternoon_2hr33min/10.20.shm


In [106]:
result = hystersis_threshold(model, meal_data,start_threshold=0.9, end_threshold=0.35, winmin = 1, stepsec=5, episode_min = 2.)

Day:  0
Segmentation Completed. 
Day:  1
Segmentation Completed. 
Day:  2
Segmentation Completed. 
Day:  3
Segmentation Completed. 


In [107]:
result

,day,segment_start,segment_end,proba,predictions,labels,segment_count
0,0,"[130, 197, 7795, 8486]","[173, 248, 7824, 8508]","[0.22874194, 0.1941067, 0.24923044, 0.19049045...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
1,1,"[17, 744, 2724, 2794, 3396, 4357, 5662, 6360, ...","[64, 798, 2767, 2856, 3464, 4392, 5733, 6439, ...","[0.4417529, 0.2943293, 0.3266197, 0.37837547, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",9
2,2,"[305, 4083, 4719, 5628, 5930, 8604, 8839, 9134]","[454, 4209, 4756, 5662, 6027, 8643, 8977, 9160]","[0.46855992, 0.35085747, 0.45749536, 0.1906475...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8
3,3,"[53, 1269, 1543, 1762]","[103, 1318, 1577, 1807]","[0.2677498, 0.27087343, 0.40227166, 0.3146106,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4


In [108]:
def print_results(result):
    from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
    total_preds = []
    total_labels = []
    result_df = {"Day":[],"Accuracy":[],"Recall":[],"AUC":[]}
    for i in range(len(result)):
        preds = result.iloc[i]['predictions']
        labels =  result.iloc[i]['labels']
        #print("Day: ", i,"Sample Num: ", len(preds) , "Prediction values: ",set(preds),"Segment Count: ",result.iloc[i]['segment_count'])

        acc = accuracy_score(preds,labels)
        rec = recall_score(preds,labels)
        auc = roc_auc_score(preds,labels)
        #print("Day-"+str(i), " Accuracy: ", acc, " Recall: ",rec,"Auc: ",auc)
        #print()
        result_df["Day"].append(str(i))
        result_df["Accuracy"].append(acc)
        result_df["Recall"].append(rec)
        result_df["AUC"].append(auc)
        total_preds.extend(preds)
        total_labels.extend(labels)

    acc= accuracy_score(total_preds,total_labels)
    rec = recall_score(total_preds,total_labels)
    auc = roc_auc_score(total_preds,total_labels)
    
    print("All Days: "," Accuracy: ", acc, " Recall: ",rec,"AUC",auc)
    result_df["Day"].append("All Days")
    result_df["Accuracy"].append(acc)
    result_df["Recall"].append(rec)
    result_df["AUC"].append(auc)
    return pd.DataFrame(result_df)


In [109]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
# time_series = np.random.random(10000)

# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5)
# meal_data = Person_MealsDataset(person_name= "wenkanw", file_name = "test_files", winmin = 1,stridesec = 5,smooth_flag = 0,
#                  normalize_flag = 0)


def f(offset,winsize,day):
    stride  = 5 *15 # 5 seconds between two adjacent labels/window samples
    
    day = int(day)
    proba = np.array(result.proba.iloc[day])
    preds = np.array(result.predictions.iloc[day])
    labels = np.array(result.labels.iloc[day])
    if winsize == -1:
        winsize = len(labels)
    
    if offset == len(labels)-winsize:
        offset = len(labels)-winsize
        
    t = np.arange(start = offset, stop= offset+winsize, step=1)
    fig, ax = plt.subplots(3,1,figsize= (20,12))
    df1 = proba[offset:offset+winsize]
    df2 = np.array(preds[offset:offset+winsize]) #*10-5
    df3 = np.array(labels[offset:offset+winsize])#*10-5
    x1= sns.lineplot(x=t, y=df1 , ax =ax[0],color= 'grey',label="Possibility")
    x2 = sns.lineplot(t,df2 , ax =ax[1],color='g', linewidth=1.5,label="Prediction(Eat)")
    x3 = sns.lineplot(t,df3 , ax =ax[2],color='b', linewidth=1.5, label="Label(Eat)")
        
    ax[0].fill_between( t, df1, 
                interpolate=True, color='grey')
    
    ax[0].fill_between(t, df3, where=(df3==1), 
                interpolate=True, color='blue')
    ax[0].fill_between( t,df2, where=(df2==1), 
                interpolate=True, color='green')
    ax[0].set_ylim(0,1)
    print(offset,len(labels)-winsize )
    if offset >= len(labels)-winsize:
        title_txt ="Day: "+ str(day) + " Whole Day samples: "+str(len(labels)) +" . " + "Sample plotted: "+str(winsize)+". "
    else:
        title_txt = "Day: "+ str(day) + "Samples from "+str(offset) +"~" + str((offset+winsize)) +". "+ "Number of Sample plotted: "+str(winsize)+". "
    ax[0].set_title(title_txt)
    ax[2].set_xlabel("index of sample")
    ax[0].set_ylabel("Possibility")
    ax[1].set_ylabel("Predictions")
    ax[2].set_ylabel("Ground Truch")
    
    ax[0].legend(["Possibility"],loc='upper left')
    ax[1].legend(["1: Eat, 0:Other"],loc='upper left')
    ax[2].legend(["1: Eat, 0:Other"],loc='upper left')
        
    return 
style = {'description_width': 'initial'}
bound_text = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(result.proba.iloc[0]),
    step=1,
    description='offset:[0~'+ str(len(result.proba.iloc[0]))+"]",
    disabled=False,
    style = {'description_width': 'initial'}
)
slidebar= widgets.IntSlider(min=0, max=len(result.proba.iloc[0]), step=1, value=0)
winsize = widgets.Dropdown(
    options=[-1, 1000],
    value=-1,
    description='samples plotted',
    disabled=False,
    style = style
)
interact(f, offset=bound_text, winsize= winsize, day = [i for i in range(len(result))]);

interactive(children=(BoundedIntText(value=0, description='offset:[0~8549]', max=8549, style=DescriptionStyle(…

In [111]:
episode_perf_df = print_results(result)
episode_perf_df

All Days:   Accuracy:  0.9530746020185205  Recall:  0.5121443442054129 AUC 0.7442073940653232


,Day,Accuracy,Recall,AUC
0,0,0.976722,0.806897,0.893275
1,1,0.952440,0.358051,0.671860
2,2,0.942971,0.603715,0.785665
3,3,0.898361,0.348315,0.652971
4,All Days,0.953075,0.512144,0.744207


In [92]:
# # np.where(result.iloc[0].labels==1)
# # x = np.array(result.labels.iloc[1]) 
# x = np.array(result.segment_start.iloc[0]) 
# np.where(x== 1)
# # x

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def f(x):
    return x
interact(f, x=True);

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

In [2]:
! jupyter --version

jupyter core     : 4.6.3
jupyter-notebook : 6.1.4
qtconsole        : 4.7.7
ipython          : 7.18.1
ipykernel        : 5.3.4
jupyter client   : 6.1.7
jupyter lab      : 2.2.6
nbconvert        : 6.0.6
ipywidgets       : 7.5.1
nbformat         : 5.0.7
traitlets        : 4.3.3


In [1]:
! jupyter --version

jupyter core     : 4.6.3
jupyter-notebook : 6.1.4
qtconsole        : 4.7.7
ipython          : 7.18.1
ipykernel        : 5.3.4
jupyter client   : 6.1.7
jupyter lab      : 2.2.6
nbconvert        : 6.0.6
ipywidgets       : 7.5.1
nbformat         : 5.0.7
traitlets        : 4.3.3
